In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
import fxcmpy
import os, sys 
import configparser
import datetime
import pandas as pd
import numpy as np
import utils

from structs import Pips, CurrencyPair
from strategy.fractals import FractalStrategy

In [3]:
abspath_api = os.path.abspath('configs/api.ini')

config = utils.ConfigHandler()

fxcm_section = config.fxcm_settings

access_token = fxcm_section['access_token']
log_file = fxcm_section['log_file']
log_level = fxcm_section['log_level']

con = fxcmpy.fxcmpy(access_token=access_token, log_level=log_level)
account_id = con.get_default_account()
print(f"FXCM API connection established")

FXCM API connection established


In [47]:
con.close()

In [4]:
RENAMER = {'bidopen': 'BidOpen', 'bidhigh': 'BidHigh', 'bidlow': 'BidLow', 'bidclose': 'BidClose',
           'askopen': 'AskOpen', 'askhigh': 'AskHigh', 'asklow': 'AskLow', 'askclose': 'AskClose'}

pair = CurrencyPair('EURUSD')
jpy_pair = pair.jpy_pair

con.subscribe_market_data(pair.fxcm_name)

In [5]:
target_level = 4.0
back_level = 2.1
break_level = Pips(2, jpy_pair)
sl_extension = Pips(1, jpy_pair)
max_width = Pips(12, jpy_pair)
min_width = Pips(2, jpy_pair)
risk = 0.015

strategy = FractalStrategy(target_level=target_level,
                           back_level=back_level,
                           break_level=break_level,
                           sl_extension=sl_extension,
                           max_width=max_width,
                           min_width=min_width,
                           risk=risk)

In [6]:
data = con.get_candles(instrument=pair.fxcm_name, period='m5', number=24)
data = data.rename(RENAMER, axis=1)

last_price = con.get_last_price(pair.fxcm_name)
latest_price = (last_price['Bid'] + last_price['Ask'])/2

upper_fractal, lower_fractal = strategy.get_fractals(data)

if upper_fractal is not None and lower_fractal is not None:
    target_l, back_l, entry_l, sl_l = strategy.get_long_order(upper_fractal, lower_fractal)
    target_s, back_s, entry_s, sl_s = strategy.get_short_order(upper_fractal, lower_fractal)
    
    if upper_fractal > latest_price > lower_fractal:
        print('Price is between fractals')
    
(upper_fractal, lower_fractal)

Price is between fractals


(1.19827, 1.197135)

In [8]:
historical_price = data

processed = pd.DataFrame()

processed['High'] = (historical_price['BidHigh'] + historical_price['AskHigh']) / 2
processed['Low'] = (historical_price['BidLow'] + historical_price['AskLow']) / 2

processed['UpperFractal'] = np.where(
    (processed['High'] > processed['High'].shift(1)) & (processed['High'] > processed['High'].shift(-1)), True,
    False)

processed['LowerFractal'] = np.where(
    (processed['Low'] < processed['Low'].shift(1)) & (processed['Low'] < processed['Low'].shift(-1)), True,
    False)

all_upper = processed[processed['UpperFractal'] == True]['High']
all_lower = processed[processed['LowerFractal'] == True]['Low']

if len(all_upper) == 0 or len(all_lower) == 0:
    None, None

upper_fractal = all_upper.iloc[-1]
lower_fractal = all_lower.iloc[-1]

### Create OCO

In [31]:
oco_order = con.create_oco_order(symbol=pair.fxcm_name, is_buy=True, is_buy2=False,
                                 amount=1, is_in_pips=False, time_in_force='GTC',
                                 rate=1.3, rate2=1.1, order_type='MarketRange', at_market=0)

### Create Entry

In [14]:
entry_order_long = con.create_entry_order(symbol=pair.fxcm_name, is_buy=True, rate=1.21, amount=1,
                                     time_in_force='GTC', is_in_pips=False, limit=1.4)

In [17]:
entry_order_long.delete()

In [20]:
entry_order_long.get_status()

'Canceled'

In [6]:
entry_order_short = con.create_entry_order(symbol=pair.fxcm_name, is_buy=False, rate=1.20700, amount=1,
                                           time_in_force='GTC', is_in_pips=False, limit=1.1)

In [10]:
entry_order_short.get_associated_trade()

'Executing'

In [12]:
return_ = entry_order_long.get_associated_trade()
return_

In [16]:
entry_order.get_status()

'Waiting'

In [18]:
return_ is None

True

### Cancel all orders

In [33]:
order_ids = con.get_order_ids()

for order_id in order_ids:
    order = con.get_order(order_id)
    order.delete()

### Open position

The order here will instantly dissapear

In [40]:
market_order = con.create_market_buy_order(symbol=pair.fxcm_name, amount=5)

In [41]:
position = market_order.get_associated_trade()

In [47]:
position.get_time()

datetime.datetime(2020, 12, 2, 14, 48, 59)

In [53]:
position.get_tradeId() in con.get_open_trade_ids()

False

In [52]:
position.close()

### Getting and closing positions

In [76]:
position_ids = con.get_open_trade_ids()

for position_id in position_ids:
    position = con.get_open_position(position_id)
    position.close()

### Change stop loss of a position

In [73]:
con.change_trade_stop_limit(trade_id=position_ids[0], is_stop=True, rate=1.33200, is_in_pips=False)

### Used margin of account

In [80]:
con.get_accounts_summary()

,t,ratePrecision,accountId,balance,usdMr,mc,mcDate,accountName,usdMr3,hedging,usableMargin3,usableMarginPerc,usableMargin3Perc,equity,usableMargin,bus,dayPL,grossPL,isTotal
0,6,0,,43365.11,0,,None,,0,,43365.11,100,100,43365.11,43365.11,None,792.53,0,True
